In [1]:
import vegafusion_jupyter as vf
import pandas as pd
import altair as alt
vf.enable()

In [2]:
source = "../data_import/data/nyc_taxi.feather"
df = pd.read_feather(source)

In [3]:
small_df = df.head(10000)

In [4]:
# df.dtypes

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_x,pickup_y,RateCodeID,store_and_fwd_flag,dropoff_x,dropoff_y,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-16 00:05:39,2015-01-16 00:23:42,1,1.59,-8.236963e+06,4.975553e+06,1,N,-8.234835e+06,4.975627e+06,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-11 01:33:38,2015-01-11 01:53:28,1,3.30,-8.237826e+06,4.971752e+06,1,N,-8.237021e+06,4.976875e+06,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-11 01:33:39,2015-01-11 01:35:31,1,0.50,-8.238654e+06,4.970221e+06,1,N,-8.238124e+06,4.971127e+06,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
3,1,2015-01-11 01:33:39,2015-01-11 01:52:58,1,3.00,-8.234434e+06,4.977363e+06,1,N,-8.238108e+06,4.974457e+06,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30
4,1,2015-01-11 01:33:39,2015-01-11 01:58:31,1,2.20,-8.235781e+06,4.972012e+06,1,N,-8.236804e+06,4.975483e+06,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30


In [6]:
width = 400
height = 400

In [7]:
base = alt.Chart(source, width=width, height=height).transform_calculate(
    pickup_hour="utchours(datum.tpep_pickup_datetime)",
    pickup_day="day(datum.tpep_pickup_datetime)",
    tip_perc="datum.tip_amount",
).transform_filter(
    "datum.tip_perc < 100"
)

In [8]:
x_domain = [-8.243204e+06, -8.226511e+06]
y_domain = [4.968192e+06, 4.982886e+06]

width_bins = 150
height_bins = 150


In [15]:
scales = alt.selection_interval(
    name="pickup_scales",
    bind='scales',
    on="[mousedown[event.altKey], window:mouseup] > window:mousemove![event.altKey]",
    translate="[mousedown[event.altKey], window:mouseup] > window:mousemove![event.altKey]",
    zoom="wheel![event.altKey]"
)

pickup_selection = alt.selection_interval(
    on="[mousedown[!event.altKey], window:mouseup] > window:mousemove![!event.altKey]",
    translate="[mousedown[!event.altKey], window:mouseup] > window:mousemove![!event.altKey]",
    zoom="wheel![!event.altKey]",
)

distance_selection = alt.selection_interval(encodings=["x"])
day_hour_selection = alt.selection_interval()


# Distance
distance_chart = base.mark_bar().encode(
    x=alt.X("trip_distance:Q", bin=alt.Bin(maxbins=20, extent=[0, 12])),
    y="count()"
).add_selection(
    distance_selection
).transform_filter(
    {"and": [
        pickup_selection,
        day_hour_selection
    ]}
)

# Pickup
pickup_chart = base.mark_rect().encode(
    x=alt.X(
        "pickup_x:Q",
        bin=alt.Bin(maxbins=width_bins),
        scale=alt.Scale(domain=x_domain),
        axis=alt.Axis(labels=False)
    ),
    y=alt.Y(
        "pickup_y:Q",
        bin=alt.Bin(maxbins=height_bins),
        scale=alt.Scale(domain=y_domain),
        axis=alt.Axis(labels=False),
    ),
    opacity=alt.Opacity("count():Q", scale=alt.Scale(type="log", range=[0.1, 1.0])),
    color=alt.value("steelblue")
    # color=alt.condition(pickup_selection, alt.value("steelblue"), alt.value("grey"))
).transform_filter(
    {"and": [
        scales,
        distance_selection,
        day_hour_selection
    ]}
).add_selection(
    scales
).add_selection(
    pickup_selection
)

# Tip percentage
tip_perc_chart = base.mark_rect().encode(
    x=alt.X(
        "pickup_day:O", 
        scale=alt.Scale(domain=[1, 2, 3, 4, 5, 6, 0]),
        axis=alt.Axis(labelExpr="datum.label==1 ? 'Mon': datum.label==0? 'Sun': ''")
    ),
    y=alt.Y("pickup_hour:O"),
    # color=alt.condition(day_hour_selection, 'mean(tip_perc):Q', alt.value('lightgray'))
    color=alt.Color('mean(tip_perc):Q'),
    opacity=alt.condition(day_hour_selection, alt.value(1.0), alt.value(0.3))
).properties(
    width=120
).add_selection(
    day_hour_selection
).transform_filter(
    {"and": [
        pickup_selection,
        distance_selection
    ]}
)


layout = distance_chart | pickup_chart | tip_perc_chart

widget = vf.VegaFusionWidget(
    layout, debounce_max_wait=None, debounce_wait=200
)
widget

VegaFusionWidget(debounce_max_wait=None, debounce_wait=200.0, spec='{\n  "config": {\n    "view": {\n      "co…